# Idea of API

See corresponding [issue](https://gitlab.kwant-project.org/semicon/semicon/issues/16).

## Basic idea

In [ ]:
from semicon.models import Zincblende
from semicon.helpers import interpolate_parameters


# coords describe space dependence of parameters
# bands and components are sequence of strings
model = Zincblende(coords='z', bands=..., components=...)


# hamiltonian property returned as sympy object
smp_ham = model.hamiltonian


# parameters returned via method of model
# this method has access to "bands" and "components" and return
# a parameter class object: subclass of dict with extra method
# for removing spurious solutions
pInAs = model.parameters(material='InAs', databank=lawaetz)
pInAs = pInAs.renormalize(gamma_0=1)



# these could be for example combined with helper function
# to provide interpolated smooth functions for a "sandwich"
# two-deg system
parameters = {k: model.parameters(material=k).renormalize(gamma_0=1) 
              for k in ['InAs', 'GaSb', 'AlSb']}

syst = ... # user defines his system of appropriate shape and fill 
           # with smp_ham through discretizer on his own

# assingment is mapping from coords to material name
parameters = interpolate_parameters(syst, parameters, assingment)

## basic bulk example

In [ ]:
from semicon.models import Zincblende

model = Zincblende(
    bands=('gamma_6c', 'gamma_8v', 'gamma_7v'),
    components=('base', 'zeeman', 'strain'),
)

params = model.parameters(material='InAs').renormalize(gamma_0=1)


# and standard kwant code (for continuum disp)

disp = kwant.continuum.lambdify(model.hamiltonian)
e_k = lambda kx, ky, kz: disp(k_x=kx, k_y=ky, k_z=kz, **params)
...


# and standard kwant code (for tb dispersion)

template = kwant.continuum.discretize(model.hamiltonian, grid_spacing=0.5)
syst = kwant.wraparound.wraparound(template).finalized()
e_k = lambda kx, ky, kz: syst.hamiltonian_submpatrix(params=dict('k_x': k_x, ..., **params))
...

## basic two-deg example

In [ ]:
from semicon.models import Zincblende
import semicon


model = Zincblende(
    coords='z',
    bands=('gamma_6c', 'gamma_8v', 'gamma_7v'),
    components=('base', 'zeeman', 'strain'),
)

parameters = {k: model.parameters(material=k).renormalize(gamma_0=1) 
              for k in ['InAs', 'GaSb', 'AlSb']}

### get system

In [ ]:
grid = 0.5
L = 20

template = kwant.continuum.discretize(model.hamiltonian, coords='z', grid_spacing=a)
syst = kwant.Builder()

shape = semicon.shapes.twodeg(start=0 - a/2, end=L + a/2)
syst.fill(template, shape, (0,))
syst = syst.finalized()

## get 2deg parameters

In [ ]:
def twodeg_mapping(z):
    """User specified mapping from coord to material."""
    return 'AlSb' if z < 5 or z > 5 else 'InAs'
 
    
pars_2deg = semicon.helpers.interpolate(
    syst=syst,
    parameters=parameters,
    mapping=twodeg_mapping 
)

## and finally obtain hamiltonian and do simulation

In [ ]:
ham = syst.hamiltonian_submatrix(params=pars_2deg)
...

# further nice helpers

## different growth direction

Basic idea about the rotation of coordinates is explained in this [notebook](./rotations.ipynb) and discussed in this [issue](https://gitlab.kwant-project.org/semicon/semicon/issues/12).

From the notebook it is clear that applying rotation produce ugly numerical coefficients in the Hamiltonian. Therefore it may be good idea to chain this method with ``prettify`` functionality.

In [ ]:
from semicon.models import Zincblende

model = Zincblende(
    bands=('gamma_6c', 'gamma_8v', 'gamma_7v'),
    components=('base', 'zeeman', 'strain'),
)

R = ... # 3x3 rotation matrix
model = model.rotate(R, act_on=semicon.symbols.momenta) \ 
             .prettify(zero_atol=1e-8, nsimplify=True)


# note: Using "act_on" to specify rotation of only momenta
# allows to leave coords unchanged (treat them as they 
# would be already defined in simulation coordinate system)